# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
import matplotlib.pyplot as plt
from convokit import Corpus, download
import random
import os
import re
import unicodedata
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import itertools
import math
import time

# Data Preprocessing

In [3]:
# Load the movie-corpus dataset
corpus = Corpus(filename=download("movie-corpus"))

In [129]:
def unicode_to_ascii(s):
    """Converts the unicode file to ascii"""
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def normalize_string(s):
    """Lowercase, trim, and remove non-letter characters"""
    # Convert to ASCII
    s = unicode_to_ascii(s.lower().strip())
    # Replace dots with nothing
    s = s.replace('.', '')
    # Space + punctuation to ensure tokens like "hello?" are treated as "hello ?"
    s = re.sub(r"([!?])", r" \1", s)
    # Remove any characters that are not a sequence of lower/upper case letters or the retained punctuation marks
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s

In [130]:
# Extract sentence pairs
all_pairs = []
for conv in corpus.get_conversation_ids(): # Iterate over each conversation
    conversation = corpus.get_conversation(conv) # Get the conversation
    utterance_ids = conversation.get_utterance_ids() # Get the utterances in the conversation
    for i in range(len(utterance_ids) - 1): # Iterate over each utterance in the conversation
        utt1 = corpus.get_utterance(utterance_ids[i]) # Get the first utterance
        utt2 = corpus.get_utterance(utterance_ids[i + 1]) # Get the second utterance
        if utt1 and utt2:  # Ensure both utterances are not None
            all_pairs.append([normalize_string(utt2.text), normalize_string(utt1.text)]) # Append the pair


In [131]:
# Flatten the list and filter out too long sentences
all_pairs = [
    pair
    for pair in all_pairs
    if 2 <= len(pair[0].split(" ")) <= 10 and 2 <= len(pair[1].split(" ")) <= 10
]

In [132]:
all_pairs[:10000]

[['they do to !', 'they do not !'],
 ['she okay ?', 'i hope so'],
 ['the real you ', 'like my fear of wearing pastels ?'],
 ['what good stuff ?', 'the real you '],
 ['i figured you d get to the good stuff eventually', 'what good stuff ?'],
 ['what crap ?', 'me this endless blonde babble i m like boring myself'],
 ['do you listen to this crap ?', 'what crap ?'],
 ['well no', 'then that s all you had to say'],
 ['you know chastity ?', 'i believe we share an art instructor'],
 ['you have my word as a gentleman', 'you re sweet'],
 ['that s because it s such a nice one', 'forget french'],
 ['c esc ma tete this is my head', 'right see ? you re ready for the quiz'],
 ['gosh if only we could find kat a boyfriend', 'let me see what i can do'],
 ['i have to be home in twenty minutes', 'i don t have to be home til two'],
 ['where did he go ? he was just here', 'who ?'],
 ['hi joey', 'you re concentrating awfully hard considering it s gym class'],
 ['hey sweet cheeks', 'hi joey'],
 ['it s more', '

In [155]:
# Split the dataset into training and validation sets and reduce the size if needed
sample_pairs = random.sample(all_pairs, int(len(all_pairs) * 1))
train_pairs, val_pairs = train_test_split(sample_pairs, test_size=0.2)

In [156]:
len(train_pairs), len(val_pairs)

(62146, 15537)

In [63]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

In [64]:
# Vocabulary class
class Voc:
    '''Class to hold the vocabulary for the dataset'''
    def __init__(self, name):
        '''Initializes the class'''
        self.name = name # Name of the vocabulary
        self.word2index = {} # Word to index dictionary
        self.word2count = {} # Word to count dictionary
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"} # Index to word dictionary
        self.num_words = 3  # Count default tokens

    def add_sentence(self, sentence):
        '''Adds a sentence to the vocabulary'''
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        '''Adds a word to the vocabulary'''
        if word not in self.word2index:
            # If the word is not in the vocabulary, add it
            self.word2index[word] = self.num_words 
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

In [157]:
# Create the vocabulary
voc = Voc('movie-corpus')

In [158]:
for pair in train_pairs:
    # Add the pair to the vocabulary
    voc.add_sentence(pair[0])
    voc.add_sentence(pair[1])

In [159]:
voc.num_words

20859

In [160]:
def filter_pairs(pairs):
    """Filters pairs to only include those with sentences that are all in the vocabulary to avoid unknown tokens"""
    filtered_val_pairs = []
    # Iterate through val_pairs
    for pair in pairs:

        # Split each sentence into words
        words_0 = pair[0].split()
        words_1 = pair[1].split()

        # Check if all words in both sentences are in the vocabulary
        if all(word in voc.word2index for word in words_0) and all(
            word in voc.word2index for word in words_1
        ):

            # If all words are in the vocabulary, append the pair to the filtered list
            filtered_val_pairs.append(pair)

In [ ]:
filtered_val_pairs = filter_pairs(val_pairs)

In [161]:
len(val_pairs), len(filtered_val_pairs)

(15537, 13753)

In [162]:
val_pairs = filtered_val_pairs # Update val_pairs to only contain pairs with sentences that are all in the vocabulary

In [14]:
def indexes_from_sentence(voc, sentence):
    '''Converts a sentence to a list of indexes based on the vocabulary'''
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zero_padding(l, fillvalue=0):
    '''Pads a list of sentences to the length of the longest sentence'''
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [16]:
def binary_matrix(l, value=0):
    """Creates a binary matrix from a list of sentences to indicate the presence of tokens"""
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [17]:
def input_var(l, voc):
    """Returns padded input sequence tensor and sequence length"""
    indexes_batch = [
        indexes_from_sentence(voc, sentence) for sentence in l
    ]  # Convert sentences to indexes
    lengths = torch.tensor(
        [len(indexes) for indexes in indexes_batch]
    )  # Create a tensor of the length of each sentence
    pad_list = zero_padding(indexes_batch)  # Pad the sentences
    pad_var = torch.LongTensor(pad_list)  # Convert to a tensor
    return pad_var, lengths

In [18]:
def output_var(l, voc):
    """Returns padded target sequence tensor, padding mask, and max target length"""
    indexes_batch = [
        indexes_from_sentence(voc, sentence) for sentence in l
    ]  # Convert sentences to indexes
    max_target_len = max(
        [len(indexes) for indexes in indexes_batch]
    )  # Get the length of the longest sentence
    pad_list = zero_padding(indexes_batch)  # Pad the sentences
    mask = binary_matrix(pad_list)  # Create a mask
    mask = torch.BoolTensor(mask)  # Convert the mask to a tensor
    pad_var = torch.LongTensor(pad_list)  # Convert the padded sentences to a tensor
    return pad_var, mask, max_target_len

In [19]:
def batch2train_data(voc, pair_batch):
    """Returns all items for a given batch of pairs"""
    pair_batch.sort(
        key=lambda x: len(x[0].split(" ")), reverse=True
    )  # Sort the pairs by the length of the input sentence
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])  # Append the input sentence to the input batch
        output_batch.append(pair[1])  # Append the output sentence to the output batch
    inp, lengths = input_var(
        input_batch, voc
    )  # Get the input tensor and the lengths of each sentence
    output, mask, max_target_len = output_var(
        output_batch, voc
    )  # Get the output tensor, mask, and the length of the longest sentence
    return inp, lengths, output, mask, max_target_len

# Model

In [21]:
class EncoderRNN(nn.Module):
    """Encoder RNN class to encode input sentences into a context vector"""

    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        """Initializes the class with the hidden size, embedding, number of layers, and dropout rate"""
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(
            hidden_size,
            hidden_size,
            n_layers,
            dropout=(0 if n_layers == 1 else dropout),
            bidirectional=True,
        )  # Use a bidirectional GRU

    def forward(self, input_seq, input_lengths, hidden=None):
        """Performs a forward pass of the encoder RNN given an input sequence and input lengths"""
        embedded = self.embedding(input_seq)  # Embed the input sequence
        packed = torch.nn.utils.rnn.pack_padded_sequence(
            embedded, input_lengths
        )  # Pack the sequence
        outputs, hidden = self.gru(
            packed, hidden
        )  # Pass the sequence and hidden state through the GRU
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(
            outputs
        )  # Unpack the sequence
        outputs = (
            outputs[:, :, : self.hidden_size] + outputs[:, :, self.hidden_size :]
        )  # Sum the bidirectional GRU outputs
        return outputs, hidden

In [22]:
class Attn(nn.Module):
    """Attention class to calculate attention weights and apply them to the encoder outputs"""

    def __init__(self, method, hidden_size):
        """Initializes the class with the attention method and hidden size"""
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ["dot", "general", "concat"]:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size

        if self.method == "general":
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == "concat":
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        """Calculates the dot product of the hidden state and the encoder output"""
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        """Performs a forward pass of the attention mechanism given the hidden state and encoder outputs"""
        attn_energies = self.dot_score(
            hidden, encoder_outputs
        )  # Calculate the attention weights (energies)
        attn_energies = attn_energies.t()  # Transpose the energies
        return F.softmax(attn_energies, dim=1).unsqueeze(
            1
        )  # Normalize the energies to probabilities

In [23]:
class LuongAttnDecoderRNN(nn.Module):
    """Luong Attn Decoder RNN class to decode the context vector and produce output sentences"""

    def __init__(
        self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1
    ):
        """Initializes the class with the attention model, embedding, hidden size, output size, number of layers, and dropout rate"""
        super(LuongAttnDecoderRNN, self).__init__()
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(
            hidden_size,
            hidden_size,
            n_layers,
            dropout=(0 if n_layers == 1 else dropout),
        )
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        """Performs a forward pass of the Luong Attn Decoder RNN given an input step, last hidden state, and encoder outputs"""
        embedded = self.embedding(input_step)  # Embed the input step
        embedded = self.embedding_dropout(
            embedded
        )  # Apply dropout to the embedded step
        rnn_output, hidden = self.gru(
            embedded, last_hidden
        )  # Pass the embedded step and last hidden state through the GRU
        attn_weights = self.attn(
            rnn_output, encoder_outputs
        )  # Calculate attention weights from the current GRU output
        context = attn_weights.bmm(
            encoder_outputs.transpose(0, 1)
        )  # Multiply attention weights by encoder outputs to get context
        rnn_output = rnn_output.squeeze(0)  # Squeeze the output
        context = context.squeeze(1)  # Squeeze the context
        concat_input = torch.cat(
            (rnn_output, context), 1
        )  # Concatenate the GRU output and context
        concat_output = torch.tanh(
            self.concat(concat_input)
        )  # Apply a tanh to the concatenated output
        output = self.out(
            concat_output
        )  # Pass the concatenated output through the output layer
        output = F.softmax(output, dim=1)  # Apply a softmax to the output
        return output, hidden

# Training

In [32]:
def maskNLLLoss(inp, target, mask):
    """Computes the negative log likelihood loss using the mask to avoid calculating the loss for PAD tokens"""
    nTotal = mask.sum()  # Get the sum of the mask
    crossEntropy = -torch.log(
        torch.gather(inp, 1, target.view(-1, 1))
    )  # Calculate the negative log likelihood loss
    loss = crossEntropy.masked_select(mask).mean()  # Apply the mask to the loss
    return loss, nTotal.item()  # Return the loss and the sum of the mask

In [33]:
def train(
    input_variable,
    lengths,
    target_variable,
    mask,
    max_target_len,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    batch_size,
    clip,
):
    """Performs a training iteration given input and target variables, a mask, the max target length, encoder, decoder, and optimizers"""
    encoder_optimizer.zero_grad()  # Zero the gradients of the encoder optimizer to avoid accumulating gradients
    decoder_optimizer.zero_grad()

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(
        input_variable, lengths
    )  # Pass the input sequence and its lengths through the encoder

    decoder_input = torch.LongTensor(
        [[SOS_token for _ in range(batch_size)]]
    )  # Create the initial decoder input (start with SOS tokens for each sentence in the batch)
    decoder_hidden = encoder_hidden[
        : decoder.n_layers
    ]  # Use the hidden state of the encoder as the initial hidden state of the decoder

    use_teacher_forcing = (
        True if random.random() < 0.5 else False
    )  # Decide if teacher forcing should be used

    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )  # Pass the decoder input, hidden state, and encoder outputs through the decoder
            decoder_input = target_variable[t].view(
                1, -1
            )  # Set the decoder input to the current target token
            mask_loss, nTotal = maskNLLLoss(
                decoder_output, target_variable[t], mask[t]
            )  # Calculate the negative log likelihood loss
            loss += mask_loss  # Add the loss
            print_losses.append(
                mask_loss.item() * nTotal
            )  # Append the loss to the list of print losses
            n_totals += nTotal  # Add the sum of the mask to the total number of tokens
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    loss.backward()  # Perform backpropagation

    _ = torch.nn.utils.clip_grad_norm_(
        encoder.parameters(), clip
    )  # Clip the gradients to avoid exploding gradients
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()  # Update the parameters of the encoder optimizer to minimize the loss
    decoder_optimizer.step()

    return sum(print_losses) / n_totals  # Return the average loss

In [84]:
def validate(encoder, decoder, voc, validation_data, batch_size):
    """Performs validation given the encoder, decoder, vocabulary, validation data, and batch size"""
    # Set the encoder and decoder model to evaluation mode
    encoder.eval()
    decoder.eval()

    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        validation_loss = 0
        n_totals = 0

        # Create batches for the validation data
        validation_batches = [
            batch2train_data(
                voc, [random.choice(validation_data) for _ in range(batch_size)]
            )
            for _ in range(len(validation_data) // batch_size)
        ]

        for validation_batch in validation_batches:
            input_variable, lengths, target_variable, mask, max_target_len = (
                validation_batch
            )

            # Forward pass through the encoder
            encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

            # Create initial decoder input (start with SOS tokens for each sentence)
            decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])

            # Set initial decoder hidden state to the encoder's final hidden state
            decoder_hidden = encoder_hidden[: decoder.n_layers]

            # Forward batch of sequences through decoder one time step at a time
            for t in range(max_target_len):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )
                # No teacher forcing: next input is decoder's own current output
                _, topi = decoder_output.topk(1)
                decoder_input = torch.LongTensor(
                    [[topi[i][0] for i in range(batch_size)]]
                )

                # Calculate and accumulate loss
                mask_loss, nTotal = maskNLLLoss(
                    decoder_output, target_variable[t], mask[t]
                )
                validation_loss += mask_loss.item() * nTotal
                n_totals += nTotal

        # Compute the average loss and perplexity over the validation set
        validation_loss_avg = validation_loss / n_totals
        perplexity = np.exp(validation_loss_avg)

    # Set the models back to training mode
    encoder.train()
    decoder.train()

    return validation_loss_avg, perplexity  # Return the average loss and perplexity

In [ ]:
def as_minutes(s):
    '''Converts seconds to minutes and seconds'''
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    '''Calculates the time since a given time and percentage of completion'''
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs)) # Return the time since and the time remaining

In [91]:
def train_iters(
    voc,
    train_pairs,
    val_pairs,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    embedding,
    n_iteration,
    batch_size,
    print_every,
    save_every,
    clip,
    validate_every,
):
    """Trains the model given the vocabulary, training pairs, validation pairs, encoder, decoder, encoder optimizer, decoder optimizer, embedding, number of iterations, batch size, print frequency, save frequency, gradient clipping, and validation frequency"""
    start = time.time()
    print_loss_total = 0
    plot_losses = []
    plot_val_losses = []
    plot_perplexities = []
    plot_val_perplexities = []

    training_batches = [
        batch2train_data(voc, [random.choice(train_pairs) for _ in range(batch_size)])
        for _ in range(n_iteration)
    ]  # Create training batches for the given number of iterations

    for iteration in range(1, n_iteration + 1):
        training_batch = training_batches[
            iteration - 1
        ]  # Get the training batch for the current iteration
        input_variable, lengths, target_variable, mask, max_target_len = (
            training_batch  # Unpack the training batch
        )

        loss = train(
            input_variable,
            lengths,
            target_variable,
            mask,
            max_target_len,
            encoder,
            decoder,
            encoder_optimizer,
            decoder_optimizer,
            batch_size,
            clip,
        )  # Perform a training iteration
        print_loss_total += loss  # Add the loss to the total loss

        if (
            iteration % print_every == 0
        ):  # If the iteration is a multiple of the print frequency
            print_loss_avg = (
                print_loss_total / print_every
            )  # Calculate the average loss
            print_loss_total = 0  # Reset the total loss
            print(
                "%s (%d %d%%) %.4f"
                % (
                    time_since(start, iteration / n_iteration),
                    iteration,
                    iteration / n_iteration * 100,
                    print_loss_avg,
                )
            )  # Print the time since the start, the percentage of completion, and the average loss
            plot_losses.append(
                print_loss_avg
            )  # Append the average loss to the list of losses
            plot_perplexities.append(
                np.exp(print_loss_avg)
            )  # Append the perplexity to the list of perplexities

        if iteration % validate_every == 0:
            val_loss, val_perplexity = validate(
                encoder, decoder, voc, val_pairs, batch_size
            )  # Perform validation
            # Append validation loss and calculate perplexity
            plot_val_losses.append(val_loss)
            plot_val_perplexities.append(val_perplexity)

        if iteration % save_every == 0:
            # Save the model and the training and validation losses and perplexities
            torch.save(
                {
                    "iteration": iteration,
                    "en": encoder.state_dict(),
                    "de": decoder.state_dict(),
                    "en_opt": encoder_optimizer.state_dict(),
                    "de_opt": decoder_optimizer.state_dict(),
                    "loss": loss,
                    "voc_dict": voc.__dict__,
                    "embedding": embedding.state_dict(),
                },
                os.path.join(
                    "checkpoints", "{}_{}.tar".format(iteration, "checkpoint")
                ),
            )

    # Plotting training and validation losses and perplexities
    print(plot_perplexities)
    print(plot_val_perplexities)
    plt.figure()
    plt.plot(plot_losses, label="Training Loss")
    plt.plot(plot_val_losses, label="Validation Loss")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(plot_perplexities, label="Training Perplexity")
    plt.plot(plot_val_perplexities, label="Validation Perplexity")
    plt.xlabel("Iteration")
    plt.ylabel("Perplexity")
    plt.legend()
    plt.show()

    return plot_losses, plot_val_losses, plot_perplexities, plot_val_perplexities

In [163]:
# Model hyperparameters
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64
decoder_learning_ratio = 5.0
clip = 50.0
learning_rate = 0.0001
n_iteration = 4000
print_every = 1
save_every = 100
validate_every = 1

In [164]:
# Initialize embeddings, encoder and decoder models
embedding = nn.Embedding(voc.num_words, hidden_size)

encoder = EncoderRNN(hidden_size, 
                     embedding, 
                     encoder_n_layers, 
                     dropout)

decoder = LuongAttnDecoderRNN('dot', 
                               embedding, 
                               hidden_size, 
                               voc.num_words,
                               decoder_n_layers, 
                               dropout)

In [165]:
# Initialize optimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

In [166]:
# Train model
train_iters(
    voc=voc,
    train_pairs=train_pairs,
    val_pairs=val_pairs,
    encoder=encoder,
    decoder=decoder,
    encoder_optimizer=encoder_optimizer,
    decoder_optimizer=decoder_optimizer,
    embedding=embedding,
    n_iteration=n_iteration,
    batch_size=batch_size,
    print_every=print_every,
    save_every=save_every,
    clip=clip,
    validate_every=validate_every
)

0m 5s (- 334m 18s) (1 0%) 9.9427
0m 38s (- 1266m 16s) (2 0%) 9.7249
1m 10s (- 1572m 41s) (3 0%) 9.5406
1m 43s (- 1722m 8s) (4 0%) 9.2533
2m 16s (- 1812m 41s) (5 0%) 8.7124
2m 49s (- 1876m 7s) (6 0%) 8.1506
3m 23s (- 1932m 2s) (7 0%) 7.6887
3m 56s (- 1963m 1s) (8 0%) 8.5029
4m 28s (- 1987m 0s) (9 0%) 8.7192
5m 1s (- 2004m 58s) (10 0%) 8.4281
5m 38s (- 2046m 31s) (11 0%) 8.4300
6m 23s (- 2126m 38s) (12 0%) 8.5958
7m 1s (- 2155m 16s) (13 0%) 8.1891
7m 37s (- 2169m 58s) (14 0%) 8.1210
8m 11s (- 2176m 56s) (15 0%) 7.5952
8m 46s (- 2183m 34s) (16 0%) 7.3705
9m 20s (- 2189m 22s) (17 0%) 7.1804
9m 53s (- 2189m 54s) (18 0%) 6.5189
10m 27s (- 2191m 2s) (19 0%) 6.3020
11m 0s (- 2189m 49s) (20 0%) 5.9895
11m 32s (- 2188m 10s) (21 0%) 6.1827
12m 5s (- 2186m 53s) (22 0%) 5.6555
12m 38s (- 2185m 0s) (23 0%) 5.6580
13m 11s (- 2184m 15s) (24 0%) 5.5057
13m 45s (- 2186m 31s) (25 0%) 5.2553
14m 18s (- 2185m 51s) (26 0%) 5.0868
14m 50s (- 2184m 24s) (27 0%) 5.2578
15m 23s (- 2182m 47s) (28 0%) 5.0065
15m 

# Evaluation

In [146]:
class GreedySearchDecoder(nn.Module):
    """Greedy search decoder class to decode the context vector and produce output sentences"""

    def __init__(self, encoder, decoder):
        """Initializes the class with the encoder and decoder models"""
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        """Performs a forward pass of the greedy search decoder given an input sequence, input length, and maximum length"""
        with torch.no_grad():
            encoder_outputs, encoder_hidden = self.encoder(
                input_seq, input_length
            )  # Pass the input sequence and its lengths through the encoder
            decoder_hidden = encoder_hidden[
                : decoder.n_layers
            ]  # Use the hidden state of the encoder as the initial hidden state of the decoder

            decoder_input = (
                torch.ones(1, 1, dtype=torch.long) * SOS_token
            )  # Create the initial decoder input (start with SOS tokens for each sentence in the batch)
            all_tokens = torch.zeros(
                [0], dtype=torch.long
            )  # Create a tensor to store the tokens of the output sentence
            all_scores = torch.zeros(
                [0]
            )  # Create a tensor to store the scores of the output sentence

            for _ in range(
                max_length
            ):  # Iterate over the maximum length of the output sentence
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )  # Pass the decoder input, hidden state, and encoder outputs through the decoder
                decoder_scores, decoder_input = torch.max(
                    decoder_output, dim=1
                )  # Get the token with the highest score
                all_tokens = torch.cat(
                    (all_tokens, decoder_input), dim=0
                )  # Concatenate the token to the list of tokens
                all_scores = torch.cat(
                    (all_scores, decoder_scores), dim=0
                )  # Concatenate the score to the list of scores
                decoder_input = torch.unsqueeze(
                    decoder_input, 0
                )  # Prepare the current token to be the next input to the decoder

            return (
                all_tokens,
                all_scores,
            )  # Return the list of tokens and the list of scores

In [147]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=10):
    """Evaluates a given sentence using the encoder, decoder, searcher, vocabulary, and maximum length"""
    indexes_batch = [
        indexes_from_sentence(voc, sentence)
    ]  # Convert the sentence to a list of indexes
    lengths = torch.tensor(
        [len(indexes) for indexes in indexes_batch]
    )  # Create a tensor of the length of the sentence
    input_batch = torch.LongTensor(indexes_batch).transpose(
        0, 1
    )  # Convert the indexes to a tensor and transpose
    tokens, scores = searcher(
        input_batch, lengths, max_length
    )  # Pass the input batch and its lengths through the searcher
    decoded_words = [
        voc.index2word[token.item()] for token in tokens
    ]  # Convert the tokens to words
    return decoded_words  # Return the list of decoded words

In [152]:
def evaluateInput(encoder, decoder, searcher, voc):
    """Evaluates user input using the encoder, decoder, searcher, and vocabulary"""
    input_sentence = ""  # Initialize the input sentence
    while True:
        try:
            input_sentence = input("> ")  # Get input sentence
            if input_sentence in ("q", "quit"):
                break  # Quit if the input is 'q' or 'quit'
            input_sentence = normalize_string(
                input_sentence
            )  # Normalize the input sentence
            output_words = evaluate(
                encoder, decoder, searcher, voc, input_sentence
            )  # Evaluate the input sentence
            output_words = [
                x for x in output_words
            ]  # if not (x == 'EOS' or x == 'PAD')
            print("Bot:", " ".join(output_words))  # Print the output sentence
        except KeyError:
            print("Error: Encountered unknown word.")

In [149]:
checkpoint = torch.load('checkpoints/50_checkpoint.tar') # Load the checkpoint

encoder_sd = checkpoint['en'] # Get the state dictionary of the encoder
decoder_sd = checkpoint['de'] # Get the state dictionary of the decoder
encoder_optimizer_sd = checkpoint['en_opt'] # Get the state dictionary of the encoder optimizer
decoder_optimizer_sd = checkpoint['de_opt'] # Get the state dictionary of the decoder optimizer
embedding_sd = checkpoint['embedding'] # Get the state dictionary of the embedding
voc.__dict__ = checkpoint['voc_dict'] # Get the vocabulary dictionary


print('Building encoder and decoder ...')


# Initialize word embeddings
embedding.load_state_dict(embedding_sd)

# Initialize encoder & decoder models
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)

print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [153]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting 
evaluateInput(encoder, decoder, searcher, voc)

Bot: EOS EOS EOS PAD PAD PAD PAD PAD PAD PAD
Bot: EOS EOS EOS PAD PAD PAD PAD PAD PAD PAD
Bot: EOS EOS EOS EOS PAD PAD PAD PAD PAD PAD
